# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

17:05:06 | NewsletterAgent.newsletter_agent | INFO | Test info message
17:05:06 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
17:05:06 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [7]:
print("🚀 Creating NewsletterAgent...")

do_download=True
process_since=None
# process_since='2025-10-05 18:30:00'

try:
    # set up state
    session_id = 'test_newsletter_20251011082816006041'
    step_name = 'step_08_draft_sections'
    del session_id, step_name
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

🚀 Creating NewsletterAgent...
session_id is not defined
test_newsletter_20251011172800612306


In [8]:
agent.state.get_status()


{'headlines': {'total': 0},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_01_fetch_urls',
  'workflow_complete': False,
  'workflow_status': 'not_started',
  'workflow_status_message': '',
  'progress_percentage': 0.0,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': True,
  'process_since': None},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [9]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


17:28:13 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Starting check_workflow_status
17:28:13 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Completed check_workflow_status
17:28:15 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Starting Step 1: Gather URLs
17:28:15 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Cleaning download/sources: 
17:28:15 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Successfully cleaned download/sources
17:28:16 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Cleaning download/text: 
17:28:16 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Successfully cleaned download/text
17:28:16 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Cleaning download/html: 
17:28:16 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Successfully cleaned download/html
2025-10-11 17:28:16,033 - fetcher_5065120144 - INFO - [fetcher_in

▶ Starting Step 1: step_01_fetch_urls


2025-10-11 17:28:16,215 - fetcher_5065120144 - INFO - [fetch_rss] RSS fetch successful for New York Times: 24 articles
2025-10-11 17:28:16,216 - fetcher_5065120144 - INFO - [fetch_html] Fetching HTML from Reddit: https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day
2025-10-11 17:28:16,310 - fetcher_5065120144 - INFO - [fetch_rss] RSS fetch successful for HackerNoon: 50 articles
2025-10-11 17:28:16,312 - fetcher_5065120144 - INFO - [fetch_html] Fetching HTML from Techmeme: https://www.techmeme.com/river
2025-10-11 17:28:16,401 - fetcher_5065120144 - INFO - [fetch_rss] RSS fetch successful for Ars Technica: 20 articles
2025-10-11 17:28:16,408 - fetcher_5065120144 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.

2025-10-11 17:28:18,074 - fetcher_5065120144 - INFO - scrape_url(https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNhbGllbmNlIjoibWVudGlvbiIsInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMifV0sImJ1bmRsZXMiOnsiYWxsRmVlZGx5TW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvY2F0ZWdvcnkvZ2xvYmFsLmFsbCJ9LHsidHlwZSI6InB1YmxpY2F0aW9uQnVja2V0IiwiaWQiOiJkaXNjb3Zlcnk6YWxsLXRvcGljcyIsInRpZXIiOiJ0aWVyMSJ9XSwiY3VzdG9tTW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvY2F0ZWdvcnkvZ2xvYmFsLmFsbCJ9XSwiYm9hcmRNb2RlIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC90YWcvZ2xvYmFsLmFsbCJ9XSwiYW5ub3RhdGVkTW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvdGFnL2dsb2JhbC5hbm5vdGF0ZWQifV19LCJyZWZpbmVNb2RlIjoiY3VzdG9tTW9kZSIsImxhbmd1YWdlcyI6WyJlbiJdfQ)
2025-10-11 17:28:18,075 - 

...

2025-10-11 17:28:42,035 - fetcher_5065120144 - INFO - Scrolling Reddit (5/5)
2025-10-11 17:28:43,769 - fetcher_5065120144 - DEBUG - performed human like actions
2025-10-11 17:28:43,829 - fetcher_5065120144 - DEBUG - Found last updated time from document.lastModified: 10/12/2025 06:28:43
2025-10-11 17:28:43,830 - fetcher_5065120144 - DEBUG - Attempting to parse last_updated: '10/12/2025 06:28:43' (type: <class 'str'>)
2025-10-11 17:28:43,830 - fetcher_5065120144 - DEBUG - Parsed datetime: 2025-10-12 06:28:43
2025-10-11 17:28:43,830 - fetcher_5065120144 - DEBUG - Added UTC timezone: 2025-10-12 06:28:43+00:00
2025-10-11 17:28:43,831 - fetcher_5065120144 - DEBUG - Converted to UTC: 2025-10-12 06:28:43+00:00
2025-10-11 17:28:43,831 - fetcher_5065120144 - DEBUG - Formatted last_updated: 2025-10-12T06:28:43Z
2025-10-11 17:28:43,831 - fetcher_5065120144 - INFO - Saving HTML to download/sources/WSJ.html
2025-10-11 17:28:43,837 - fetcher_5065120144 - INFO - [fetch_html] Parsing HTML file: downlo

,source,url
0,Ars Technica,20
1,Bloomberg,31
2,Business Insider,15
3,FT,46
4,Feedly AI,68
5,Hacker News,28
6,HackerNoon,50
7,New York Times,24
8,NewsAPI,82
9,Reddit,50


,source,title,url,published,rss_summary,id
0,Ars Technica,OpenAI will stop saving most ChatGPT users’ de...,https://arstechnica.com/tech-policy/2025/10/op...,"Fri, 10 Oct 2025 14:58:29 +0000",Court ends controversial order forcing OpenAI ...,0
1,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,1
2,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",2
3,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,3
4,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,4
...,...,...,...,...,...,...
744,NewsAPI,US stocks drop after Trump threatens more tari...,https://www.wyff4.com/article/us-stocks-drop-w...,2025-10-10T18:48:19Z,NaN,744
745,NewsAPI,US stocks drop after Trump threatens more tari...,https://www.4029tv.com/article/us-stocks-drop-...,2025-10-10T18:55:28Z,NaN,745
746,NewsAPI,US stocks drop after Trump threatens more tari...,https://www.koco.com/article/us-stocks-drop-wo...,2025-10-10T18:50:53Z,NaN,746
747,NewsAPI,US stocks drop after Trump threatens more tari...,https://www.wtae.com/article/us-stocks-drop-wo...,2025-10-10T18:49:49Z,NaN,747


17:28:50 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Completed Step 1: Gathered 860 articles
17:28:52 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Starting check_workflow_status
17:28:52 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Completed check_workflow_status


⏱️  Total execution time: 44.90s
📊 Final result:
Step 1 (Gather URLs) completed.

What I did:
- Collected headlines and URLs from RSS sources.
- Stored results in persistent state.

Results:
- Sources scanned: 17 (RSS)
- Articles gathered: 860 fetched, 749 stored in state

Current workflow status:
- Step 1: complete
- Next step: Step 2 — Filter URLs to AI-related content

Would you like me to run Step 2 (filter URLs) now, or stop here?


In [24]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,...,bt_z,adjusted_len,rating,bradley_terry,topics,extended_summary,cluster,cluster_name,cat,dupe_id
0,Feedly AI,"Analysis: in 2025, tech companies have raised ...",https://www.bloomberg.com/news/newsletters/202...,NaN,NaN,0,True,skipped,https://www.bloomberg.com/news/newsletters/202...,,...,2.988516,0.000000,9.893164,3.816463,[],"Analysis: in 2025, tech companies have raised ...",-1,Other,AI Market Risks,-1.0
1,Feedly AI,Interviews with security researchers about AI'...,https://www.nytimes.com/2025/10/10/opinion/ai-...,NaN,NaN,1,True,200,https://www.nytimes.com/2025/10/10/opinion/ai-...,download/html/Interviews_with_security_researc...,...,3.024335,0.151676,9.868403,4.001566,"[AI Doom, AI Risks, AI Governance, Expert Opin...",Interviews with security researchers about AI'...,3,AI Infrastructure and Environmental Impact,Existential AI Risk Debate,-1.0
2,Feedly AI,It’s not too late for Apple to get AI right,https://techcrunch.com/2025/10/11/its-not-too-...,NaN,NaN,3,True,200,https://techcrunch.com/2025/10/11/its-not-too-...,download/html/It_s_not_too_late_for_Apple_to_g...,...,2.580266,1.076640,8.302341,1.706730,"[Virtual Assistants, Chatbots, App Platforms, ...",It’s not too late for Apple to get AI right\n\...,-1,Other,AI Media Impact,-1.0
3,Feedly AI,"Ready or not, enterprises are betting on AI",https://techcrunch.com/2025/10/11/ready-or-not...,NaN,NaN,5,True,200,https://techcrunch.com/2025/10/11/ready-or-not...,download/html/Ready_or_not__enterprises_are_be...,...,3.124409,0.751972,8.663886,4.518722,"[Enterprise AI, Customer Service, AI Challenge...","Ready or not, enterprises are betting on AI\n\...",1,"AI Investment, Hype, and Market Risks",Enterprise AI Deals,-1.0
4,Feedly AI,'We're kind of in a fragile state now': Why th...,https://finance.yahoo.com/news/were-kind-fragi...,NaN,NaN,6,True,200,https://finance.yahoo.com/news/were-kind-fragi...,download/html/We_re_kind_of_in_a_fragile_state...,...,2.944469,0.839164,8.474838,3.588836,"[AI Bubble, Market Trends, Stocks, Economics, ...",'We're kind of in a fragile state now': Why th...,1,"AI Investment, Hype, and Market Risks",AI Market Risks,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,NewsAPI,Klarna CEO Says AI Will Drive ‘Massive Shift’ ...,https://biztoc.com/x/7d9b0e407c1f82bc,2025-10-10T19:24:51Z,NaN,73,True,200,https://biztoc.com/x/7d9b0e407c1f82bc,download/html/Klarna_CEO_Says_AI_Will_Drive__M...,...,1.882049,0.082426,3.299990,-1.901474,"[Workforce Transformation, Job Automation, Ban...",Klarna CEO Says AI Will Drive ‘Massive Shift’ ...,1,"AI Investment, Hype, and Market Risks",Other,-1.0
61,NewsAPI,How Neurosymbolic AI Finds Growth That Others ...,https://hbr.org/sponsored/2025/10/how-neurosym...,2025-10-10T19:57:37Z,NaN,74,True,200,https://hbr.org/sponsored/2025/10/how-neurosym...,download/html/How_Neurosymbolic_AI_Finds_Growt...,...,1.797617,0.725503,3.026744,-2.337801,"[Neurosymbolic AI, Explainable AI, Regulated I...",How Neurosymbolic AI Finds Growth That Others ...,1,"AI Investment, Hype, and Market Risks",LLM Hallucinations,-1.0
62,Feedly AI,"SEMI: US chip fab investment to outpace China,...",https://asia.nikkei.com/business/tech/semicond...,NaN,NaN,75,True,200,https://asia.nikkei.com/business/tech/semicond...,download/html/SEMI__US_chip_fab_investment_to_...,...,3.921698,0.000000,5.121540,8.638909,"[Semiconductor Investment, US Chip Manufacturi...","SEMI: US chip fab investment to outpace China,...",0,Global Tensions Impacting Semiconductor Supply...,Semiconductor Supply Chain Tensions,-1.0
63,Feedly AI,Pilot Medicare program gives AI a say in appro...,https://www.expressnews.com/business/article/m...,NaN,NaN,76,True,403,https://www.expressnews.com/business/article/m...,,...,2.822758,0.000000,3.822755,2.959866,[],Pilot Medicare program gives AI a say in appro...,-1,Other,AI Legal Impact,-1.0


In [25]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
1,Feedly AI,31
2,NewsAPI,29
3,Reddit,2
0,Business Insider,1
4,Techmeme,1
5,VentureBeat,1


In [12]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


17:28:55 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Starting Step 2: Filter URLs
17:28:55 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | 🔍 Filtering 749 headlines...
17:28:55 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
17:28:55 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | 🔍 Filtering 749 articles for dupes.


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/tech-policy/2025/10/openai-no-longer-forced-to-save-deleted-chats-but-some-users-still-affected/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/ai-models-can-acquire-backdoors-from-surprisingly-few-malicious-documents/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/bank-of-england-warns-ai-stock-bubble-rivals-2000-dotcom-peak/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/tech-policy/2025/10/vandals-deface-ads-for-ai-necklaces-that-listen-to-all-your-conversations/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/insurers-balk-at-paying-out-huge-settlements-for-claims-ag

checking https://www.johndcook.com/blog/2025/10/06/a-quiet-change-to-rsa/
url not found - inserting new URL
checking https://blog.ui.com/article/all-new-next-gen-of-unifi-storage
url not found - inserting new URL
checking https://arstechnica.com/gadgets/2025/10/people-regret-buying-amazon-smart-displays-after-being-bombarded-with-ads/
url not found - inserting new URL
checking https://fulghum.io/album-cards
found url, no cutoff set - treating as duplicate
checking https://www.reuters.com/sustainability/boards-policy-regulation/indonesia-says-22-plants-industrial-zone-near-jakarta-contaminated-by-caesium-2025-10-08/


17:28:56 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | URL deduplication: 559 duplicates filtered, 190 new URLs remain
17:28:56 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | 🔍 Filtering 749 headlines for AI relevance using LLM...
INFO:llm:Initialized LangfuseClient


url not found - inserting new URL
checking https://cruzgodar.com/applets/wilsons-algorithm/
url not found - inserting new URL
checking https://neilmadden.blog/2025/09/12/rating-26-years-of-java-changes/
url not found - inserting new URL
checking https://zerophilosophy.substack.com/p/crypto-current
url not found - inserting new URL
checking https://zayenz.se/blog/post/how-to-check-for-overlapping-intervals/
url not found - inserting new URL
checking https://ericmigi.com/blog/re-introducing-the-pebble-appstore/
found url, no cutoff set - treating as duplicate
checking https://alexandrepoupeau.com/otary/api/image/transformers/thresholding/
url not found - inserting new URL
checking https://james-simon.github.io/blog/chicken-cooking/
found url, no cutoff set - treating as duplicate
checking https://news.sky.com/story/discord-hack-shows-dangers-of-online-age-checks-as-internet-policing-hopes-put-to-the-test-13447618
url not found - inserting new URL
checking https://blog.tangled.org/intro
f

INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954


concurrency:  16


17:28:56 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Processing 8 chunks with concurrency 16
17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Completed Step 2: 82 AI-related articles, 559 duplicates removed


⏱️  Total execution time: 12.74s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 559 duplicate URLs, classified 190 new articles, found 82 AI-related.


In [13]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Starting Step 3: Download Articles
17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Starting concurrent scraping of 82 AI-related articles


▶ Starting Step 3: step_03_download_articles


17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Launching browser for 82 URLs with 16 concurrent workers
17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Worker 0 fetching 1 of 82 https://hackaday.com/2025/10/10/your-llm-wont-stop-lying-any-time-soon/
17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | scrape_url(https://hackaday.com/2025/10/10/your-llm-wont-stop-lying-any-time-soon/)
17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | scraping https://hackaday.com/2025/10/10/your-llm-wont-stop-lying-any-time-soon/ to download/html
17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Downloading https://hackaday.com/2025/10/10/your-llm-wont-stop-lying-any-time-soon/
17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Worker 1 fetching 2 of 82 https://financialpost.com/investing/companies-overpaying-ai-stock-market-bubble-survey
17:29:08 | NewsletterAgent.t

17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Worker 11 fetching 10 of 82 https://www.the-independent.com/arts-entertainment/films/news/robin-williams-daughter-zelda-ai-videos-b2840650.html
17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | scrape_url(https://www.the-independent.com/arts-entertainment/films/news/robin-williams-daughter-zelda-ai-videos-b2840650.html)
17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | scraping https://www.the-independent.com/arts-entertainment/films/news/robin-williams-daughter-zelda-ai-videos-b2840650.html to download/html
17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Downloading https://www.the-independent.com/arts-entertainment/films/news/robin-williams-daughter-zelda-ai-videos-b2840650.html
17:29:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Worker 12 fetching 11 of 82 https://siliconangle.com/2025/10/10/sustainable-ai-computing-rewi

17:29:10 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | scraping https://www.globenewswire.com/news-release/2025/10/10/3165081/32719/en/AI-Lead-Plaintiff-Deadline-Reminder-Shareholders-Who-Want-to-Lead-the-Class-Action-Against-C3-ai-Inc-Should-Contact-Robbins-LLP-Before-October-21-2025.html to download/html
17:29:10 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Downloading https://www.globenewswire.com/news-release/2025/10/10/3165081/32719/en/AI-Lead-Plaintiff-Deadline-Reminder-Shareholders-Who-Want-to-Lead-the-Class-Action-Against-C3-ai-Inc-Should-Contact-Robbins-LLP-Before-October-21-2025.html
17:29:10 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Response: 200
17:29:11 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Response: 200
17:29:11 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Response: 200
17:29:11 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Response: 200
17:29:12 | New

...

concurrency:  16


17:43:49 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Inserted 2 new site records
17:43:49 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Updated 82 URL records with final URLs


Starting with 82 rows...
Processing 82 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Getting embeddings for 76 texts...


17:43:53 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Completed Step 3: Downloaded 76 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
Removing 0 rows due to high similarity 
Final dataframe has 82 rows (removed 0 rows)
⏱️  Total execution time: 885.26s
📊 Final result:
✅ Step 3 step_03_download_articles completed successfully! Downloaded 76 AI-related articles with 93% success rate.
📊 Average article length: 5818 characters
🔗 Content stored in persistent state.


In [14]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["status"]!=200])




,id,source,title,status,final_url
76,155,Feedly AI,"OpenAI’s Sora used to make deepfake AI videos of dead celebrities, outraging their families",403,https://www.fastcompany.com/91420771/openais-sora-used-to-make-deepfake-ai-videos-of-dead-celebrities-outraging-their-families
77,164,Feedly AI,Pilot Medicare program gives AI a say in approving or denying treatment,403,https://www.expressnews.com/business/article/medicare-ai-prior-authorization-texas-21091940.php
78,176,Feedly AI,"Analysis: in 2025, tech companies have raised about $157B in the US bond markets, up 70% from last year, as debt seeps into every corner of the AI economy (Edward Ludlow/Bloomberg)",skipped,https://www.bloomberg.com/news/newsletters/2025-10-06/big-debt-deals-throw-fuel-on-the-ai-boom
79,161,Feedly AI,Microsoft Sees Healthcare as Path to Independence From OpenAI - Tech News Briefing - WSJ Podcasts,skipped,https://www.wsj.com/podcasts/tech-news-briefing/microsoft-sees-healthcare-as-path-to-independence-from-openai/e2bfc8d7-44c0-44e1-8f0d-34821630f82e
80,154,Feedly AI,Tariffs and AI Spending in Focus as Traders Brace For Earnings,skipped,https://www.bloomberg.com/news/articles/2025-10-11/tariffs-and-ai-spending-in-focus-as-traders-brace-for-earnings
81,123,Feedly AI,Thinking Machines Lab Co-Founder Departs for Meta,skipped,https://www.wsj.com/tech/ai/thinking-machines-lab-co-founder-departs-for-meta-442d7461


In [15]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])



,id,source,title,status,final_url
76,155,Feedly AI,"OpenAI’s Sora used to make deepfake AI videos of dead celebrities, outraging their families",403,https://www.fastcompany.com/91420771/openais-sora-used-to-make-deepfake-ai-videos-of-dead-celebrities-outraging-their-families
77,164,Feedly AI,Pilot Medicare program gives AI a say in approving or denying treatment,403,https://www.expressnews.com/business/article/medicare-ai-prior-authorization-texas-21091940.php
78,176,Feedly AI,"Analysis: in 2025, tech companies have raised about $157B in the US bond markets, up 70% from last year, as debt seeps into every corner of the AI economy (Edward Ludlow/Bloomberg)",skipped,https://www.bloomberg.com/news/newsletters/2025-10-06/big-debt-deals-throw-fuel-on-the-ai-boom
79,161,Feedly AI,Microsoft Sees Healthcare as Path to Independence From OpenAI - Tech News Briefing - WSJ Podcasts,skipped,https://www.wsj.com/podcasts/tech-news-briefing/microsoft-sees-healthcare-as-path-to-independence-from-openai/e2bfc8d7-44c0-44e1-8f0d-34821630f82e
80,154,Feedly AI,Tariffs and AI Spending in Focus as Traders Brace For Earnings,skipped,https://www.bloomberg.com/news/articles/2025-10-11/tariffs-and-ai-spending-in-focus-as-traders-brace-for-earnings
81,123,Feedly AI,Thinking Machines Lab Co-Founder Departs for Meta,skipped,https://www.wsj.com/tech/ai/thinking-machines-lab-co-founder-departs-for-meta-442d7461


In [16]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

17:43:53 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Processing 82 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1273, user_len=43
17:43:53 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Using model 'gpt-4.1-mini' for summarization
17:43:53 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Starting summarization for 82 articles


▶ Starting Step 4: step_04_extract_summaries
concurrency:  16


17:43:53 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Processing 82 chunks with concurrency 16
17:44:09 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Extracting metadata from HTML files for 82 articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/item_distiller' from Langfuse
INFO:llm:Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, system_len=1365, user_len=151
17:44:12 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Processing 82 chunks with concurrency 16


concurrency:  16
⏱️  Total execution time: 90.44s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 82/82 articles.
💾 Summaries stored in headline DataFrame.


In [17]:
# if we get a refusal, examine prompt and delete rows
bad_stuff =["What\'s the Best Way ",
            "AI could make it eas",
            "AI can design toxic ",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20, 20, 20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags,short_summary


In [ ]:
# remove bad stuf
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [18]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


17:45:24 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Rating 82 AI articles using fn_rate_articles
17:45:24 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Calculating article ratings for 82 articles
17:45:24 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Rating recency
17:45:24 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Rating spam probability
17:45:24 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Initialized LangfuseClient
17:45:24 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
17:45:24 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 5: step_05_rate_articles
concurrency:  16


17:45:27 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | low quality articles: {0.0: 66, 1.0: 5, 0.09534945969074979: 2, 8.423463754468647e-12: 1, 1.994729895832688e-06: 1, 1.026187963170189e-10: 1, 2.1087728885906676e-13: 1, 6.914400106940203e-13: 1, 0.9999832372181827: 1, 0.9999998063873693: 1, 0.999933888949188: 1}
17:45:27 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
17:45:27 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240


concurrency:  16


17:45:30 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | on topic articles: {1.0: 63, 0.9999998063873693: 2, 0.9999884821053749: 1, 0.9947799085613173: 1, 5.619689396832023e-15: 1, 0.9241417710874382: 1, 0.9999545100305701: 1, 3.7266389560521204e-06: 1, 0.7310585784564795: 1, 0.0011695103074966034: 1, 0.9999723899264568: 1, 2.9989608248573072e-09: 1, 0.999999448776502: 1, 0.9999832372181827: 1, 0.9999995679801056: 1, 0.022977367413042835: 1, 7.484619074298232e-05: 1, 0.00017952801482389944: 1}
17:45:30 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Rating importance probability
17:45:30 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Successfully retrieved prompt 'newsagent/rate_importance' from Langfuse
17:45:30 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Parsed prompt 'newsagent/rate_importance': model=gpt-4.1, system_len=2145, user_len=252


...

,id,site_name,title,bradley_terry,bt_z
55,55,Nextgov,AI export control bill passes Senate as NDAA amendment,10.489914,1.353542
51,51,Live Mint,The World’s Chip Supply Chain Is Bracing for Fallout From China’s Rare-Earth Curbs,9.205673,1.187833
45,45,The Motley Fool,AMD Soars on OpenAI Deal. Is It Too Late to Buy the Stock?,8.749394,1.128958
26,26,Tom's Hardware,"Microsoft deploys world's first 'supercomputer-scale' GB300 NVL72 Azure cluster — 4,608 GB300 GPUs linked together to form a single, unified accelerator capable of 92.1 exaFLOPS of FP4 inference",8.745904,1.128508
75,75,Nikkei Asia,"SEMI: US chip fab investment to outpace China, Taiwan, and South Korea from 2027, driven by AI demand and US policies, rising from $21B in 2025 to $43B in 2028 (Nikkei Asia)",8.731588,1.126660
...,...,...,...,...,...
72,72,Entertainment Weekly,Sara Haines hangs head as Joy Behar mistakes AI-generated woman for “The View” cohost Sunny Hostin: 'That's Sunny',-17.296180,-2.231773
17,17,The Wall Street Journal,Microsoft Sees Healthcare as Path to Independence From OpenAI - Tech News Briefing - WSJ Podcasts,-17.725141,-2.287123
33,33,Bloomberg,Tariffs and AI Spending in Focus as Traders Brace For Earnings,-18.533340,-2.391407
16,16,The Wall Street Journal,Thinking Machines Lab Co-Founder Departs for Meta,-19.189609,-2.476087


17:51:50 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Top 10 ids: [55, 51, 45, 26, 75, 24, 2, 46, 19, 37]
17:51:50 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | After round 9/16: 
17:51:50 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Number of ranking changes: 32
17:51:50 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Sum of absolute ranking changes: 918.0 (avg rank chg 11.33)
17:51:50 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | last_two: 10.691358024691358, prev_two: 15.777777777777779
17:51:50 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | ---------------------------------------------------
17:51:50 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Running round 10 of max 16
17:51:50 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | ---------------------------------------------------
17:51:50 | NewsletterAgent.test_newsletter_20251011172800612306 |

,id,site_name,title,bradley_terry,bt_z
55,55,Nextgov,AI export control bill passes Senate as NDAA amendment,10.485305,1.352660
75,75,Nikkei Asia,"SEMI: US chip fab investment to outpace China, Taiwan, and South Korea from 2027, driven by AI demand and US policies, rising from $21B in 2025 to $43B in 2028 (Nikkei Asia)",8.638909,1.114465
51,51,Live Mint,The World’s Chip Supply Chain Is Bracing for Fallout From China’s Rare-Earth Curbs,8.638132,1.114365
26,26,Tom's Hardware,"Microsoft deploys world's first 'supercomputer-scale' GB300 NVL72 Azure cluster — 4,608 GB300 GPUs linked together to form a single, unified accelerator capable of 92.1 exaFLOPS of FP4 inference",8.635101,1.113974
45,45,The Motley Fool,AMD Soars on OpenAI Deal. Is It Too Late to Buy the Stock?,8.593344,1.108587
...,...,...,...,...,...
72,72,Entertainment Weekly,Sara Haines hangs head as Joy Behar mistakes AI-generated woman for “The View” cohost Sunny Hostin: 'That's Sunny',-17.413675,-2.246457
17,17,The Wall Street Journal,Microsoft Sees Healthcare as Path to Independence From OpenAI - Tech News Briefing - WSJ Podcasts,-17.799229,-2.296196
33,33,Bloomberg,Tariffs and AI Spending in Focus as Traders Brace For Earnings,-18.603502,-2.399951
16,16,The Wall Street Journal,Thinking Machines Lab Co-Founder Departs for Meta,-19.258802,-2.484489


17:52:35 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Top 10 ids: [55, 75, 51, 26, 45, 24, 2, 46, 19, 37]
17:52:35 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | After round 10/16: 
17:52:35 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Number of ranking changes: 53
17:52:35 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Sum of absolute ranking changes: 1038.0 (avg rank chg 12.81)
17:52:35 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | last_two: 12.074074074074074, prev_two: 11.123456790123457
17:52:35 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Increase in avg rank change, stopping
17:52:35 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Low rated articles: 3
17:52:35 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | low rated article: Small reactors, big problems: the nuclear mirage behind AI’s energy hype -0.1532340759044417
17:52:35 | NewsletterA

⏱️  Total execution time: 432.90s
📊 Final result:
✅ Step 5 step_05_rate_articles completed successfully! Rated 78 articles with average rating 6.0/10.
⭐ High quality articles (≥7.0): 28
💾 Ratings stored in persistent state.


In [19]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
0,Bloomberg,"Analysis: in 2025, tech companies have raised about $157B in the US bond markets, up 70% from last year, as debt seeps into every corner of the AI economy (Edward Ludlow/Bloomberg)",9.893164,"Bloomberg analysis by Edward Ludlow finds tech companies raised about $157 billion in U.S. bond markets in 2025, up 70% from 2024, as debt extends across the AI economy."
1,The New York Times,"Interviews with security researchers about AI's potential for large-scale destruction, as experts remain divided and global regulatory frameworks lag (Stephen Witt/New York Times)",9.868403,"The New York Times reports that in 2024 AI pioneer Yoshua Bengio warned AI could engineer lethal pathogens posing existential risk, while Meta's Yann LeCun dismissed such risks, underscoring expert division and lagging global governance."
2,Gizmodo,The Destruction in Gaza Is What the Future of AI Warfare Looks Like,9.610850,"An article says Israel has used AI systems since 2021 to identify and target individuals and infrastructure in Gaza, while major U.S. tech firms supply cloud, AI and surveillance services, prompting employee protests and international human-rights criticism."
7,Yahoo,China's lesson for the US: it takes more than chips to win the AI race,8.825128,"Alibaba announced a roadmap toward artificial superintelligence, integrating Qwen open-source models with its cloud to become a full-stack AI provider, as US and Chinese tech firms collectively invest over $400 billion in AI infrastructure, with US firms leading spending."
8,Yahoo,Stacy Rasgon Says Nvidia (NVDA)-Open AI Deal Shows There’s ‘Shortage of Compute’ – ‘Customers Are Lining Up Years in Advance’,8.722789,"Bernstein analyst Stacy Rasgon says Nvidia's $100 billion OpenAI deal highlights a compute shortage as customers prebook years ahead; in Q2 FY2026 Nvidia earned $41.3B of $46.7B from AI infrastructure concentrated among few clients, raising overcapacity and China-export risks."
...,...,...,...,...
74,Harvard Business Review,How Neurosymbolic AI Finds Growth That Others Cannot See - SPONSOR CONTENT FROM EY-PARTHENON,3.026744,"EY-Parthenon says neurosymbolic AI, combining neural networks' pattern recognition with symbolic reasoning, reduces complex analyses like tariff-impact assessments from months to a day, helps prevent generative-AI hallucinations, and provides explainable, traceable decisions for regulated industries."
33,Bloomberg,Tariffs and AI Spending in Focus as Traders Brace For Earnings,2.650250,Traders are focusing on tariffs and corporate AI spending as they prepare for upcoming earnings reports.
70,Forbes,5 Ways To Measure AI ROI That Actually Work For Small Businesses,1.305334,"Small businesses can measure AI ROI by tracking five areas: cost savings, direct sales impact, productivity gains, customer satisfaction and retention, and decision-making improvements; assign financial values and clear success criteria to evaluate tools' measurable business impact."
77,Biztoc,Prediction: These 4 Artificial Intelligence (AI) Stocks Will Be Worth More Than $7 Trillion by 2030,0.487707,"Analysts predict four leading AI stocks will together exceed $7 trillion in market value by 2030, driven by expected AI-driven innovation and deal-making that could reshape tech industry valuations and competitive dynamics."


In [20]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


9.9 - [Analysis: in 2025, tech companies have raised about $157B in the US bond markets, up 70% from last year, as debt seeps into every corner of the AI economy (Edward Ludlow/Bloomberg)](https://www.bloomberg.com/news/newsletters/2025-10-06/big-debt-deals-throw-fuel-on-the-ai-boom) - Bloomberg

Bloomberg analysis by Edward Ludlow finds tech companies raised about $157 billion in U.S. bond markets in 2025, up 70% from 2024, as debt extends across the AI economy.
    


9.9 - [Interviews with security researchers about AI's potential for large-scale destruction, as experts remain divided and global regulatory frameworks lag (Stephen Witt/New York Times)](https://www.nytimes.com/2025/10/10/opinion/ai-destruction-technology-future.html) - The New York Times

The New York Times reports that in 2024 AI pioneer Yoshua Bengio warned AI could engineer lethal pathogens posing existential risk, while Meta's Yann LeCun dismissed such risks, underscoring expert division and lagging global governance.
    


9.6 - [The Destruction in Gaza Is What the Future of AI Warfare Looks Like](https://gizmodo.com/the-destruction-in-gaza-is-what-the-future-of-ai-warfare-looks-like-2000669559) - Gizmodo

An article says Israel has used AI systems since 2021 to identify and target individuals and infrastructure in Gaza, while major U.S. tech firms supply cloud, AI and surveillance services, prompting employee protests and international human-rights criticism.
    


8.8 - [China's lesson for the US: it takes more than chips to win the AI race](https://finance.yahoo.com/news/chinas-lesson-us-takes-more-093000938.html) - Yahoo

Alibaba announced a roadmap toward artificial superintelligence, integrating Qwen open-source models with its cloud to become a full-stack AI provider, as US and Chinese tech firms collectively invest over $400 billion in AI infrastructure, with US firms leading spending.
    


8.7 - [Stacy Rasgon Says Nvidia (NVDA)-Open AI Deal Shows There’s ‘Shortage of Compute’ – ‘Customers Are Lining Up Years in Advance’](https://finance.yahoo.com/news/stacy-rasgon-says-nvidia-nvda-205736797.html) - Yahoo

Bernstein analyst Stacy Rasgon says Nvidia's $100 billion OpenAI deal highlights a compute shortage as customers prebook years ahead; in Q2 FY2026 Nvidia earned $41.3B of $46.7B from AI infrastructure concentrated among few clients, raising overcapacity and China-export risks.
    

...


3.0 - [How Neurosymbolic AI Finds Growth That Others Cannot See - SPONSOR CONTENT FROM EY-PARTHENON](https://hbr.org/sponsored/2025/10/how-neurosymbolic-ai-finds-growth-that-others-cannot-see) - Harvard Business Review

EY-Parthenon says neurosymbolic AI, combining neural networks' pattern recognition with symbolic reasoning, reduces complex analyses like tariff-impact assessments from months to a day, helps prevent generative-AI hallucinations, and provides explainable, traceable decisions for regulated industries.
    


2.7 - [Tariffs and AI Spending in Focus as Traders Brace For Earnings](https://www.bloomberg.com/news/articles/2025-10-11/tariffs-and-ai-spending-in-focus-as-traders-brace-for-earnings) - Bloomberg

Traders are focusing on tariffs and corporate AI spending as they prepare for upcoming earnings reports.
    


1.3 - [5 Ways To Measure AI ROI That Actually Work For Small Businesses](https://www.forbes.com/sites/terdawn-deboe/2025/10/10/5-ways-to-measure-ai-roi-that-actually-work-for-small-businesses/) - Forbes

Small businesses can measure AI ROI by tracking five areas: cost savings, direct sales impact, productivity gains, customer satisfaction and retention, and decision-making improvements; assign financial values and clear success criteria to evaluate tools' measurable business impact.
    


0.5 - [Prediction: These 4 Artificial Intelligence (AI) Stocks Will Be Worth More Than $7 Trillion by 2030](https://biztoc.com/x/3329997a1fd2afd0) - Biztoc

Analysts predict four leading AI stocks will together exceed $7 trillion in market value by 2030, driven by expected AI-driven innovation and deal-making that could reshape tech industry valuations and competitive dynamics.
    


0.3 - [Sara Haines hangs head as Joy Behar mistakes AI-generated woman for “The View” cohost Sunny Hostin: 'That's Sunny'](https://ew.com/joy-behar-mistakes-ai-generated-woman-sunny-hostin-the-view-11827324) - Entertainment Weekly

On The View, Joy Behar mistakenly identified an AI-generated image by Hasan Minhaj—depicting him and two women at a fictitious Hot Topics table—as cohost Sunny Hostin, prompting laughter and discussion about AI's impacts.
    

In [21]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


17:52:37 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
17:52:37 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Using model 'gpt-4.1-mini' for topic extraction
17:52:37 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Processing 78 articles for topic extraction
17:52:37 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Processing 8 chunks with concurrency 16


▶ Starting Step 6: step_06_cluster_by_topic
concurrency:  16


17:52:42 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Successfully extracted 312 total topics across articles
17:52:42 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Starting canonical topic classification for 101 topics
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/can

INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized Langfu

INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_top

...

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/topic_writer' from Langfuse
INFO:llm:Parsed prompt 'newsagent/topic_writer': m

=== HDBSCAN Parameters ===
min_cluster_size:   4
min_samples:        2
n_components:       20
=== Clustering Quality Metrics ===
Number of clusters: 5
Noise points: 42 (53.8%)
Average cluster size: 7.2 ± 2.9
Cluster size range: 4 - 12
=== Quality Scores ===
Silhouette Score: 0.278 (higher is better)
Calinski-Harabasz Score: 8.6 (higher is better)
Davies-Bouldin Score: 1.289 (lower is better)
HDBSCAN Validity Index: 0.091
Composite Score: 0.185 (higher is better)

[I 2025-10-11 17:53:19,262] Trial 199 finished with value: -0.18451742575763297 and parameters: {'n_components': 20, 'min_cluster_size': 4, 'min_samples': 2}. Best is trial 163 with value: -0.21320534736969338.

Optimization completed!
Best composite score: 0.2132
Best parameters: {'n_components': 21, 'min_cluster_size': 3, 'min_samples': 2}

=== Results with Best Parameters ===
Reduced dimensions from 3072 to 21
=== Clustering Quality Metrics ===
Number of clusters: 4
Noise points: 37 (47.4%)
Average cluster size: 10.2 ± 6.5


17:53:20 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | 1: AI Investment, Hype, and Market Risks
17:53:20 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Ready or not, enterprises are betting on AI (Enterprise AI, Customer Service, AI Challenges, AI Partnerships, Deals, Job Automation, Legal Issues)
'We're kind of in a fragile state now': Why the AI bubble might be about to burst — how to protect yourself (AI Bubble, Market Trends, Stocks, Economics, Investment Risks, Finance, Gen AI)
Stacy Rasgon Says Nvidia (NVDA)-Open AI Deal Shows There’s ‘Shortage of Compute’ – ‘Customers Are Lining Up Years in Advance’ (Nvidia, AI Infrastructure, Infrastructure, Deals, Stocks, Funding, Gen AI)
At SF Tech Week, the AI 'gold rush' shows no signs of slowing down (AI Investment, Venture Capital, AI Startups, Tech Events, Economics, Finance, Gen AI)
Andreessen Horowitz-Backed FurtherAI Raises $25M To Give Insurance Professionals 'Superpowers' Through Workflow Automatio

17:53:22 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | 



⏱️  Total execution time: 44.88s
📊 Final result:
✅ Step 6 step_06_cluster_by_topic completed successfully! Organized 78 articles into topic clusters.


In [22]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


17:53:22 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Free form categorization of articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, system_len=638, user_len=1179


▶ Starting Step 7: step_07_select_sections


17:54:10 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Cleaning up initial categories: ['Global Tensions Impacting Semiconductor Supply Chain', 'China Full-Stack AI Push', 'Existential AI Risk Divide', 'LLM Hallucination Challenges', 'Square AI Tools For Restaurants', 'AI Infrastructure and Environmental Impact', 'AI Investment, Hype, and Market Risks', 'AI Warfare And Tech Support', 'AMD OpenAI GPU Deal', 'OpenAI Ad Platform Plans', 'Figure 03 Humanoid Robot', 'AI Market Bubble Risks', 'Enterprise AI Deal Wave', 'Hyperscale Compute Race', 'Sustainable AI Data Centers', "AI's Impact on Media and Law", 'Nvidia Market Spotlight', 'Ant Ling-1T Model', 'OpenAI Data Retention Ruling', 'Optical Interconnects For AI', 'FurtherAI Series A Funding', 'Other', 'Circular AI Mega-Deals Concern', 'China Nvidia Import Crackdown', 'Hollywood-AI Clash', 'AI Inference Benchmarking']
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_cleanup' from 

17:55:18 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Handling singleton categories
17:55:18 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Singleton categories (0): []
17:55:18 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Non-singleton categories (14): ['AI Media Impact', 'AI Market Risks', 'Other', 'Enterprise AI Deals', 'Nvidia', 'AI Legal Impact', 'Sustainable AI Data Centers', 'Hyperscale Compute Race', 'Semiconductor Supply Chain Tensions', 'Existential AI Risk Debate', 'Hollywood AI Clash', 'AI Inference Benchmarking', 'Optical Interconnects For AI', 'LLM Hallucinations']


⏱️  Total execution time: 116.01s
📊 Final result:
Categories and article counts:
cat
AI Inference Benchmarking               2
AI Legal Impact                         5
AI Market Risks                         9
AI Media Impact                        10
Enterprise AI Deals                     7
Existential AI Risk Debate              2
Hollywood AI Clash                      2
Hyperscale Compute Race                 3
LLM Hallucinations                      2
Nvidia                                  5
Optical Interconnects For AI            2
Other                                   8
Semiconductor Supply Chain Tensions     3
Sustainable AI Data Centers             5


In [23]:
# User prompt to run workflow
# user_prompt = "Run step 8, draft sections"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


17:55:18 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Step 8a: Selecting stories from 65 total
17:55:18 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | 10 must-include stories with rating > 8
17:55:18 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | 55 stories with rating <= 8


▶ Starting Step 8: step_08_draft_sections


17:55:21 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Selected 55 diverse Tier 2 stories via MMR
17:55:21 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Total selected stories: 65 (target: 100)
17:55:21 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Step 8b: Drafting sections for 14 categories (including Other) from selected stories
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5, system_len=1854, user_len=22
17:56:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Drafted 14 sections, now flattening to DataFrame
17:56:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Reassigned 4 pruned stories to Other section
17:56:08 | NewsletterAgent.test_newsletter_20251011172800612306 | INFO | Created newsletter_section_df with 65 stories across 14 categories


⏱️  Total execution time: 50.35s
📊 Final result:
Drafted 65 stories across 14 sections


In [6]:
# User prompt to run workflow
# user_prompt = "Run step 9, finalize newsletter"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("finalize_newsletter")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


17:05:20 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Step 9a: Critiquing and optimizing individual sections
17:05:20 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Moving 2 singleton categories to Other: ['ChatGPT Bias Reduction', 'OpenAI Safety Issues']
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/critique_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/critique_section': model=gpt-5, system_len=1142, user_len=123
17:05:20 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Processing 17 sections
17:05:20 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Initialized LLMagent:
system_prompt:   You are an expert newsletter editor specializing in technology news curation. Your task is to critique individual newsletter sections
  and provide actionable recommendations to improve quality, coherence, and readability.

  For each section, you will:
  1. Assess thematic coherence -

▶ Starting Step 9: step_09_finalize_newsletter


17:05:21 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | User message: **Section Title:** Other News

**Available target_category values**: 
['Workforce AI Impact\nOpenAI Deal Network\nOpenAI Governance And Legal\nSam Altman AI Expansion\nCross-Sector AI Adoption\nLLM Reasoning Pretraining\nAI Cloud Infrastructure Surge\nBanking AI Impact\nHigh-Bandwidth Memory Growth\nLLM Inference Benchmarking\nTech Firms AI Hiring\nSoftware Development AI Tools\nAI Investment Growth\nAI Startup Funding\nAI Emerging Risks\nSemantic Search Across Data']

**Headlines:**
[{'id': 41, 'headline': 'AI video app piece lacks summary.', 'rating': 3.12589485, 'links': '[The Wall Street Journal](https://www.wsj.com/tech/personal-tech/i-tried-the-hot-new-ai-video-app-it-made-me-lonelier-than-ever-c9fdcceb)'}, {'id': 6, 'headline': "OpenAI touts GPT-5's 30% political-bias reduction after 100-topic stress test; adds tone controls and guidelines, as studies flag persistent biases across other AI.", 

17:05:21 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | User message: **Section Title:** AI Levels Up Reasoning

**Available target_category values**: 
['Workforce AI Impact\nOther\nOpenAI Deal Network\nOpenAI Governance And Legal\nSam Altman AI Expansion\nCross-Sector AI Adoption\nAI Cloud Infrastructure Surge\nBanking AI Impact\nHigh-Bandwidth Memory Growth\nLLM Inference Benchmarking\nTech Firms AI Hiring\nSoftware Development AI Tools\nAI Investment Growth\nAI Startup Funding\nAI Emerging Risks\nSemantic Search Across Data']

**Headlines:**
[{'id': 64, 'headline': 'Nvidia unveils RLP training that prompts LLMs to generate reasoning chains first, lifting reasoning benchmarks up to 17% and stabilizing enterprise fine-tuning.', 'rating': 8.0229878524, 'links': '[VentureBeat](https://venturebeat.com/ai/nvidia-researchers-boost-llms-reasoning-skills-by-getting-them-to-think)'}, {'id': 84, 'headline': 'Google DeepMind debuts Gemini Robotics 1.5 and ER 1.5, enabling robot

17:05:21 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | User message: **Section Title:** AI Talent Arms Race

**Available target_category values**: 
['Workforce AI Impact\nOther\nOpenAI Deal Network\nOpenAI Governance And Legal\nSam Altman AI Expansion\nCross-Sector AI Adoption\nLLM Reasoning Pretraining\nAI Cloud Infrastructure Surge\nBanking AI Impact\nHigh-Bandwidth Memory Growth\nLLM Inference Benchmarking\nSoftware Development AI Tools\nAI Investment Growth\nAI Startup Funding\nAI Emerging Risks\nSemantic Search Across Data']

**Headlines:**
[{'id': 83, 'headline': 'TCS doubles AI workforce to 160,000, plans $5–6B for 1GW AI data centers; 150MW first phase in 14–18 months.', 'rating': 6.4453312441, 'links': '[The Economic Times](https://m.economictimes.com/markets/expert-view/tcs-ai-skilled-workforce-doubles-to-1-6-lakh-18500-new-hires-in-q2-with-focus-on-future-ready-skills/articleshow/124453937.cms)'}, {'id': 94, 'headline': 'TCS opens AI Experience Zone and Des

...

17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |     Pruned 0 stories of 84, leaving 84
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |       MOVE id=85 to AI Emerging Risks: Focuses on metric inflation and environmental accounting, which fits risk/impact more than infrastructure buildout.
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |     Pruned 0 stories of 84, leaving 84
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |       Coherence: 7.5/10,       Quality: 7.0/10
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |       REWRITE id=89: Anchors the section with credible regulatory focus; tighten for clarity and brevity. Consider sourcing from FSB/BIS release or Reuters for authority.
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |         Old: FSB, BIS tighten AI risk oversight in finance, warn of shared-model systemic threats, cyber and fr

17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |         New: TCS opens London AI hub and design studio, targets 5,000 UK jobs in three years.
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |     Pruned 0 stories of 84, leaving 84
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |       REWRITE id=26: Clear acqui-hire aligns with talent arms race; shorten and clarify.
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |         Old: Apple nears acqui-hire of Prompt AI to fold Seemour vision tech into Apple Intelligence; Seemour app to be retired.
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |         New: Apple to acqui-hire Prompt AI, integrate Seemour vision tech into Apple Intelligence.
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |     Pruned 0 stories of 84, leaving 84
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | 

17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |     Pruned 1 stories of 81, leaving 82
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |       DROP id=106: Portfolio composition story is weakly connected to AI strategy; low-authority source; adds little to the narrative.
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |     Pruned 1 stories of 80, leaving 81
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |       DROP id=124: Rating < 3.0 and generic ETF roundup; not aligned with strategic power plays.
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |     Pruned 1 stories of 79, leaving 80
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |       Coherence: 9.0/10,       Quality: 8.0/10
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |       REWRITE id=55: Strong fit and source; headline is long and cluttered. Lead with investo

17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |         New: AI-generated homeless-man TikTok triggers false 911 calls in U.S. and U.K.
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |     Pruned 0 stories of 78, leaving 78
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |       MOVE id=92 to Other: Governance/ethics of political figure advising AI firms fits better outside this theme and lacks strong sourcing.
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |     Pruned 0 stories of 78, leaving 78
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |       DROP id=75: Generic opinion piece with low authority and limited specifics; dilutes section focus.
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |     Pruned 1 stories of 77, leaving 78
17:06:37 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO |       DROP id=80: Niche parenting work

⏱️  Total execution time: 82.48s
📊 Final result:
❌ Step 9 failed: RetryError[<Future at 0x12d991b10 state=finished raised Exception>]


In [ ]:
newsletter_section_df = state.newsletter_section_df.copy()


In [ ]:
cat_df = newsletter_section_df.groupby(["cat", "section_title"]).agg({
    'rating': 'mean',  # average rating per category
    'id': 'count'      # story count
}).rename(columns={'id': 'count'}).sort_values('rating', ascending=False).reset_index()


In [ ]:
            # Move singleton categories to "Other"
            singleton_cats = cat_df[cat_df['count'] == 1]['cat'].tolist()
            if singleton_cats:
                logger.info(
                    f"Moving {len(singleton_cats)} singleton categories to Other: {singleton_cats}")
                for singleton_cat in singleton_cats:
                    newsletter_section_df.loc[newsletter_section_df['cat']
                                              == singleton_cat, 'cat'] = 'Other'
                    newsletter_section_df.loc[newsletter_section_df['cat']
                                              == 'Other', 'section_title'] = 'Other News'


In [ ]:
                cat_df = newsletter_section_df.groupby(["cat", "section_title"]).agg({
                    'rating': 'mean',
                    'id': 'count'
                }).rename(columns={'id': 'count'}).reset_index()


In [ ]:
cat_df_dict = dict(zip(cat_df['cat'], cat_df['section_title']))
cat_df_dict


In [ ]:
            unique_cats = newsletter_section_df['cat'].unique()
            logger.info(f"Processing {len(unique_cats)} sections")


In [ ]:
print("\n".join(unique_cats))

In [ ]:

class SectionItemAction(BaseModel):
    """Action to take on a specific story within a section"""
    id: int = Field(description="Story ID from newsletter_section_df")
    action: str = Field(
        description="Action: 'keep', 'drop', 'rewrite', 'move'"
    )
    reason: str = Field(description="Why this action is recommended")
    rewritten_headline: Optional[str] = Field(
        default=None,
        description="New headline text if action=='rewrite'"
    )
    target_category: Optional[str] = Field(
        default=None,
        description="Target category name if action=='move'"
    )


class SectionCritique(BaseModel):
    """Quality critique for a single newsletter section"""
    section_name: str = Field(description="Category name being critiqued")
    section_title: str = Field(description="Section title")

    overall_coherence: float = Field(
        description="0-10: How well stories fit together thematically"
    )
    overall_quality: float = Field(
        description="0-10: Overall section quality (considering ratings, headlines, coherence)"
    )

    should_split: bool = Field(
        description="True if section is too heterogeneous and should be split"
    )
    split_recommendation: Optional[str] = Field(
        default=None,
        description="Explanation of how to split if should_split==True"
    )

    item_actions: List[SectionItemAction] = Field(
        description="Recommended action for each story in section",
        default_factory=list
    )

    summary_notes: str = Field(
        description="Overall assessment of section strengths and weaknesses"
    )

    should_iterate: bool = Field(
        description="True if changes needed and section should be re-critiqued"
    )


class OptimizedSection(BaseModel):
    """Optimized section after applying critique recommendations"""
    section_name: str = Field(description="Category name")
    section_title: str = Field(description="Section title (may be updated)")
    stories: List[Dict[str, Any]] = Field(
        description="List of story dicts with keys: id, headline, rating, links"
    )



# Newsletter critique models for quality evaluation

class DuplicateIssue(BaseModel):
    """Identified duplicate or near-duplicate story across sections"""
    headline_1: str = Field(description="First headline text")
    section_1: str = Field(description="Section containing first headline")
    headline_2: str = Field(description="Second headline text")
    section_2: str = Field(description="Section containing second headline")
    explanation: str = Field(description="Why these are considered duplicates")


class HeadlineIssue(BaseModel):
    """Quality issue with a specific headline"""
    headline: str = Field(description="The problematic headline")
    section: str = Field(description="Section containing this headline")
    issue_type: str = Field(
        description="Type of issue: too_long, passive_voice, unclear, missing_specifics, jargon"
    )
    suggestion: str = Field(
        description="Specific suggestion to improve this headline")

class SectionIssue(BaseModel):
    """Quality issue with a section"""
    section_title: str = Field(description="Title of the problematic section")
    issue_type: str = Field(
        description="Type of issue: too_small, too_large, incoherent, title_mismatch"
    )
    suggestion: str = Field(
        description="Specific suggestion to improve this section")


class NewsletterCritique(BaseModel):
    """Comprehensive quality evaluation of newsletter draft"""
    overall_score: float = Field(
        description="Overall quality score 0-10 (9-10 excellent, 8-9 good, 7-8 acceptable, <7 needs work)"
    )

    # Specific issues (empty lists if none found)
    duplicate_issues: List[DuplicateIssue] = Field(
        default_factory=list,
        description="List of duplicate or near-duplicate stories found"
    )
    headline_issues: List[HeadlineIssue] = Field(
        default_factory=list,
        description="List of headline quality issues"
    )
    section_issues: List[SectionIssue] = Field(
        default_factory=list,
        description="List of section quality issues"
    )

    # Dimension scores (0-10 each)
    theme_coherence: float = Field(
        description="0-10: How well H1 title reflects content and sections cluster thematically"
    )
    headline_quality: float = Field(
        description="0-10: Clarity, conciseness, specificity, active voice"
    )
    source_quality: float = Field(
        description="0-10: Use of authoritative sources (Reuters, Bloomberg, FT, etc.)"
    )
    format_compliance: float = Field(
        description="0-10: Adherence to markdown format rules"
    )

    # Actionable feedback
    recommendations: List[str] = Field(
        description="Top 3-5 specific, actionable improvements needed",
        default_factory=list
    )
    should_iterate: bool = Field(
        description="True if score < 8.5 and issues are fixable through iteration"
    )

In [ ]:
            critique_agent = LLMagent(
                system_prompt=section_critique_system,
                user_prompt=section_critique_user,
                output_type=SectionCritique,
                model=section_critique_model,
                verbose=True,
                logger=logger
            )


In [ ]:
            async def critique_wrapper(cat):
                cat_stories = newsletter_section_df.loc[newsletter_section_df['cat'] == cat]
                section_title = cat_stories['section_title'].iloc[0]
                section_input = cat_stories[[
                    'id', 'headline', 'rating', 'links']].to_dict('records')
                critique = await critique_agent.run_prompt(section_title=section_title, target_categories=str("\n".join(unique_cats)), input_text=section_input)
                return (cat, critique)

In [ ]:
            tasks = [critique_wrapper(cat) for cat in unique_cats]

            critiques = await asyncio.gather(*tasks)


In [ ]:
critiques 


In [ ]:

            logger.info(f"      Coherence: {critiques[0][1].overall_coherence:.1f}/10, "
                             f"Quality: {critiques[0][1].overall_quality:.1f}/10")

In [ ]:
newsletter_section_df['prune']=False
for cat, critique in critiques:
    for action in critique.item_actions:
        story_mask = newsletter_section_df['id'] == action.id
        if action.action == 'drop':
            logger.info(
                f"      DROP id={action.id}: {action.reason}")
            newsletter_section_df.loc[story_mask,
                                      'prune'] = True
            newsletter_section_df.loc[story_mask,
                                      'cat'] = 'Other'
            newsletter_section_df.loc[story_mask,
                                      'section_title'] = 'Other News'
            changes_made = True
        elif action.action == 'rewrite' and action.rewritten_headline:
            old_headline = newsletter_section_df.loc[story_mask,
                                                     'headline'].iloc[0]
            logger.info(
                f"      REWRITE id={action.id}: {action.reason}")
            logger.info(f"        Old: {old_headline}")
            logger.info(
                f"        New: {action.rewritten_headline}")
            newsletter_section_df.loc[story_mask,
                                      'headline'] = action.rewritten_headline
            changes_made = True
            
        elif action.action == 'move' and action.target_category:
            logger.info(
                f"      MOVE id={action.id} to {action.target_category}: {action.reason}")
            newsletter_section_df.loc[story_mask,
                                      'cat'] = action.target_category
            newsletter_section_df.loc[story_mask,
                                      'section_title'] = cat_df_dict[action.target_category]
            
#             catchbad cat
            changes_made = True


In [ ]:
print(len(newsletter_section_df))
newsletter_section_df = newsletter_section_df.loc[~newsletter_section_df['prune']]
print(len(newsletter_section_df))


In [ ]:
x = [v for c,v in critiques if c=="AI Cloud Infrastructure Surge"][0]
x 



In [ ]:
pd.DataFrame([(xx.id, xx.action, xx.reason, xx.rewritten_headline, xx.target_category) for xx in x.item_actions])

In [ ]:
sections_md = []
for _, row in cat_df.iterrows():
    cat = row['cat']
    section_title = row['section_title']

    # Get stories for this category, sorted by rating (descending)
    cat_stories = newsletter_section_df[
        newsletter_section_df['cat'] == cat
    ].sort_values('rating', ascending=False)

    # Build markdown section
    section_md = f"## {section_title}\n\n"
    for _, story in cat_stories.iterrows():
        section_md += f"- {story['headline']} - {story['links']}\n"

    sections_md.append(section_md)

In [ ]:
display(Markdown("\n\n".join(sections_md)))

In [ ]:
                    embeddings = embedding_df.values


In [ ]:
target_tier2 = 100 - len(must_include)
target_tier2

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def mmr_selection(
    df: pd.DataFrame,
    embeddings: np.ndarray,
    n: int = 100,
    lambda_param: float = 0.5
) -> pd.DataFrame:
    """
    Select diverse, high-quality stories using Max Marginal Relevance.

    Balances story quality (rating) with diversity (embedding similarity)
    to avoid redundant coverage of the same story angles.

    Args:
        df: DataFrame with 'rating' column and matching embedding indices
        embeddings: numpy array of embeddings (shape: [len(df), embedding_dim])
        n: Number of stories to select
        lambda_param: Tradeoff between relevance (rating) and diversity
                     1.0 = pure rating, 0.0 = pure diversity, 0.5 = balanced

    Returns:
        DataFrame subset of n selected stories with maximum rating and diversity
    """
    if len(df) <= n:
        return df

    # Normalize ratings to 0-1 scale
    max_rating = df['rating'].max()
    min_rating = df['rating'].min()
    rating_range = max_rating - min_rating

    if rating_range == 0:
        normalized_ratings = pd.Series([0.5] * len(df), index=df.index)
    else:
        normalized_ratings = (df['rating'] - min_rating) / rating_range

    selected_indices = []
    remaining_indices = list(df.index)

    # Start with highest rated story
    first_idx = df['rating'].idxmax()
    selected_indices.append(first_idx)
    remaining_indices.remove(first_idx)

    # Get embedding index mapping (df.index might not be 0..n-1)
    idx_to_embedding_pos = {idx: pos for pos, idx in enumerate(df.index)}

    while len(selected_indices) < n and remaining_indices:
        mmr_scores = []

        for idx in remaining_indices:
            # Relevance: normalized rating
            relevance = normalized_ratings.loc[idx]

            # Diversity: 1 - max_similarity to already selected
            embedding_pos = idx_to_embedding_pos[idx]
            selected_positions = [idx_to_embedding_pos[i]
                                  for i in selected_indices]

            similarities = cosine_similarity(
                embeddings[embedding_pos:embedding_pos+1],
                embeddings[selected_positions]
            )
            max_similarity = similarities.max()
            diversity = 1 - max_similarity

            # Combined MMR score
            mmr = lambda_param * relevance + (1 - lambda_param) * diversity
            mmr_scores.append((idx, mmr))

        # Select highest MMR
        best_idx = max(mmr_scores, key=lambda x: x[1])[0]
        selected_indices.append(best_idx)
        remaining_indices.remove(best_idx)

    return df.loc[selected_indices]


In [ ]:
                    tier2_selected = mmr_selection(
                        df=candidates,
                        embeddings=embeddings,
                        n=target_tier2,
                        lambda_param=0.5  # 50% rating, 50% diversity
                    )


In [ ]:
type(must_include)



In [ ]:

# Combine tiers
selected_df = pd.concat(
    [must_include, tier2_selected])
logger.info(
    f"Total selected stories: {len(selected_df)} (target: ~100-120)")
selected_df

In [ ]:
            headline_df = selected_df.copy()


In [ ]:
            # Get unique categories
            categories = headline_df['cat'].unique().tolist()
            categories = [cat for cat in categories if cat != "Other"]


In [ ]:
class SectionStoryLink(BaseModel):
    url: str = Field(description="URL of the article")
    site_name: str = Field(description="Name of the website/source")

    def __str__(self):
        return f"[{self.site_name}]({self.url})"


class SectionStory(BaseModel):
    headline: str = Field(description="Summary of the story")
    links: List[SectionStoryLink] = Field(
        description="List of links related to this story")
    prune: bool = Field(description="Whether to prune/exclude this story")

    def __str__(self):
        return f"- {self.headline} - " + " ".join([str(s) for s in self.links])


class Section(BaseModel):
    section_title: str = Field(description="Title of the newsletter section")
    headlines: List[SectionStory] = Field(
        description="List of stories in this section")

    def __str__(self):
        return f"## {self.section_title}\n\n" + "\n".join(
            [str(h) for h in self.headlines if not h.prune]
        )



In [ ]:
            write_section_system_prompt, write_section_user_prompt, model = \
                LangfuseClient().get_prompt("newsagent/write_section")

            write_section_agent = LLMagent(
                system_prompt=write_section_system_prompt,
                user_prompt=write_section_user_prompt,
                output_type=Section,
                model=model,
                verbose=True,
                logger=logger
            )


In [ ]:
            async def draft_section(cat, agent):
                """Draft a section for a given category"""
                # Get articles for this category, sorted by rating, convert to JSON
                cat_df = headline_df.loc[headline_df["cat"] == cat].sort_values(
                    "rating", ascending=False)

                input_text = cat_df[["rating", "short_summary", "site_name", "final_url"]].rename(columns={"short_summary": "summary", "final_url": "url"}).to_json(
                    orient="records")

                # Call the LLM to draft the section
                response = await agent.run_prompt(input_text=input_text)

                return (cat, response)

In [ ]:
            # Draft all sections asynchronously
            draft_tasks = [draft_section(cat, write_section_agent)
                           for cat in categories]
            draft_results = await asyncio.gather(*draft_tasks, return_exceptions=True)


In [ ]:
draft_results[0]


In [ ]:
            sections_drafted = 0
            for result in draft_results:
                if isinstance(result, Exception):
                    self.logger.error(f"Error drafting section: {result}")
                    continue

                cat, content = result
                # state.newsletter_section_obj[cat] = content
                state.newsletter_section_text[cat] = content
                sections_drafted += 1

In [ ]:
for k, v in state.newsletter_section_text.items():
    display(Markdown(str(v).replace("$", "\\\$")))


In [ ]:
cat_df = state.headline_df.groupby("cat") \
    .count() \
    .reset_index()[['cat','source']] \
    .sort_values('source', ascending=False)
output_str = ""
for cat in cat_df["cat"]:
    if cat != "Other":
        output_str += str(state.newsletter_section_text[cat]) + "\n\n"
        display(Markdown(str(state.newsletter_section_text[cat]).replace("$", "\\\$")))

# first do full rewrite .
# check vs. objects , not showing the ones market for pruning
# move prune=True to Other

In [ ]:
print(output_str)


In [ ]:
draft_newsletter_system_prompt, draft_newsletter_user_prompt, model = \
    LangfuseClient().get_probmpt("newsagent/draft_newsletter")


In [ ]:
class Mystr(BaseModel):
    """A string"""
    mystr: str = Field(
        description="a string")


In [ ]:
draft_newsletter_agent = LLMagent(
    system_prompt=draft_newsletter_system_prompt,
    user_prompt=draft_newsletter_user_prompt,
    output_type=Mystr,
    model=model,
    verbose=True,
    logger=logger
)


In [ ]:
# Apply prompt to generate final newsletter
newsletter_content = await draft_newsletter_agent.run_prompt(input_str=output_str)


In [ ]:
newsletter_content = newsletter_content.mystr
display(Markdown(newsletter_content))


In [ ]:
HTML(newsletter_content)

In [ ]:
%pip install markdown 
from utilities import send_gmail
import markdown

In [ ]:
newsletter_content_html = markdown.markdown(newsletter_content)

In [ ]:
                today = datetime.now().strftime("%B %d, %Y")
                subject = f"AI News Digest - {today}"

                # Apply HTML styling
                html_content = f"""
                <div style="max-width: 800px; margin: 0 auto; font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial, sans-serif;">
                    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 40px 20px; text-align: center; border-radius: 8px 8px 0 0;">
                        <h1 style="color: white; margin: 0; font-size: 32px;">AI News Digest</h1>
                        a<p style="color: rgba(255,255,255,0.9); margin: 10px 0 0 0; font-size: 16px;">{today}</p>
                    </div>
                    <div style="background: #ffffff; padding: 30px; border-radius: 0 0 8px 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
                        {newsletter_content_html}
                    </div>
                    <div style="text-align: center; padding: 20px; color: #666; font-size: 14px;">
                        <p>Generated on {today} by AI Newsletter Agent</p>
                    </div>
                </div>
                """

                send_gmail(subject, html_content)

In [ ]:
use mmr 
do the pruning 
rewrite other after pruning
need to pass thinking effort
critic optimizer loop

In [ ]:
input_text = headline_df.loc[headline_df["cat"]=="AI Business Value Gap"].sort_values("rating", ascending=False)[["rating", "short_summary", "site_name", "url"]].to_json(orient="records")
input_text


In [ ]:
write_section_system_prompt, write_section_user_prompt, model = \
    LangfuseClient().get_prompt("newsagent/write_section")

write_section_agent = LLMagent(
    system_prompt=write_section_system_prompt,
    user_prompt=write_section_user_prompt,
    output_type=Section,
    model=model,
    verbose=True,
    logger=logger
)


In [ ]:
response = await write_section_agent.run_prompt(input_text=input_text)
response

In [ ]:
display(Markdown(str(response)))

In [ ]:
class Link(BaseModel):
    url: str = Field(description="URL of the article")
    site_name: str = Field(description="Name of the website/source")
    def __str__(self):
        return f"[{self.site_name}]({self.url})"


class Story(BaseModel):
    headline: str = Field(description="Summary of the story")
    links: List[Link] = Field(description="List of links related to this story")
    prune: bool = Field(description="Whether to prune/exclude this story")
    def __str__(self):
        return "" if self.prune else f"- {self.headline} - " + " ".join([str(s) for s in self.links])


class Section(BaseModel):
    section_title: str = Field(description="Title of the newsletter section")
    headlines: List[Story] = Field(description="List of stories in this section")
    def __str__(self):
        return f"## {self.section_title}\n\n" + "\n".join(
            [str(h) for h in self.headlines]
        )
        

In [ ]:
headline_df=state.headline_df
headline_df.loc[headline_df["cat"]=="AI Business Value Gap"].sort_values("rating", ascending=False)[["rating", "short_summary", "site_name", "url"]].to_json(orient="records")


In [ ]:
print("# SUGGESTED TOPICS:")
catcount = headline_df.groupby("cat").count().reset_index()[['cat', 'source']].sort_values('source', ascending=False)
for c in catcount["cat"]:
    print(c)
print()
print("# RAW NEWS ITEMS:")
i =0
for row in headline_df.sort_values(["cat", "rating"], ascending=False).itertuples():
    print(f"[{row.title}]({row.url}) - {row.site_name}\n".replace("$","\\\\$"))
    row_topics = ", ".join(row.topics)
    print(f"Topics: {row_topics}\n".replace("$","\\\\$"))
    print(f"Rating: {row.rating:.1f}\n")    
    print(f"{row.short_summary}\n".replace("$","\\\\$"))
    print(f"{row.summary}\n".replace("$","\\\\$"))
    print("~~~\n")
    i +=1


In [ ]:
headline_df['cat'].unique() 


In [ ]:
astate.get_completed_steps() 



In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
headline_df=state.headline_df
i =0
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"{row.rating:.1f}"))   
    display(Markdown(f"[{row.title}]({row.url}) - {row.site_name}".replace("$","\\\\$")))
    row_topics = ", ".join(row.topics)
    display(Markdown(f"Topics: {row_topics}".replace("$","\\\\$")))
    display(Markdown(f"{row.short_summary}".replace("$","\\\\$")))
    display(Markdown(f"{row.summary}".replace("$","\\\\$")))
    i +=1
#     if i>=30:
#         break

In [ ]:
class DistilledStory(BaseModel):
    """DistilledStory class for structured output distillation into a single sentence """
    item: str = Field(description="List of StoryRating")
        
system, user, model = LangfuseClient().get_prompt("newsagent/item_distiller")

distill_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=DistilledStory,
            model=model,
            verbose=False,
            logger=logger
        )

In [ ]:
response = await distill_agent.run_prompt(input_text="""AI 'Homeless Man' Challenge Sparks Outrage as Police Called Over Dangerous Viral Trend - International Business Times

Topics: AI Pranks, Public Safety, Ethical Concerns, Disinformation, Policy And Regulation, Snapchat Challenges, Gen AI

Rating: 1.5

Topics: AI Pranks, Public Safety, Ethical Concerns, Disinformation, Policy And Regulation, Snapchat Challenges, Gen AI

Parents in shock after dangerous 'homeless man' Snapchat AI prank goes horribly wrong, police called to calm the viral chaos.

A viral AI-driven prank called the 'homeless man' challenge on Snapchat deceived parents into thinking a homeless person had broken into their home, prompting police intervention.
The prank caused significant public backlash due to the misuse of AI technology to create real emergency scares, leading to police questioning the pranksters and debates about the legal consequences.
Experts emphasize the ethical concerns and potential emotional harm caused by such digital pranks, highlighting the need for responsible use of AI to avoid wasting emergency resources and creating community panic.
""")
response

In [ ]:
headline_df=state.headline_df
i =0
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    print(f"[{row.title}]({row.url}) - {row.site_name}\n".replace("$","\\\\$"))
    row_topics = ", ".join(row.topics)
    print(f"Topics: {row_topics}\n".replace("$","\\\\$"))
    print(f"{row.short_summary}\n".replace("$","\\\\$"))
    print(f"{row.summary}\n".replace("$","\\\\$"))
    print("~~~\n")

    i +=1

TODO:
- update final prompt
- output sections using short summary
# SUGGESTED TOPICS
AI Agents And Reliability
AI Creative Industry Impact
AI Development Tools And Standards
AI Market Valuations
AI Phishing Surge
AI Security Risks
AI Workforce Impact
C2PA Image Provenance
Circular Deal Inflation
Cross-Industry AI Adoption
Crunch Lab Decentralized AI
Data Center Environmental Impact
Deepfake Video Ethics
Dell Raises AI Forecasts
EU AI Strategy
Enterprise AI Data Leakage
Enterprise AI Partnerships
Google Gemini 2.5
Healthcare AI Investments
OpenAI Platform Issues
Other
Qualcomm Acquires Arduino
SoftBank Acquires ABB Robotics
Youth Support AI Ethics

- take each summary and boil it down to 1 sentence , output correct format
- initial write sections - prompt and output json for each section asynchronously
- check and rewrite each section for format asynchronsously
- assemble sections
- do a critic loop